In [58]:
from cfg.configfile_req import *
from redminelib import Redmine

redmine = Redmine(req_server_url,key=req_key_txt)
projects = redmine.project.all()

print("Proyectos:")
for p in projects:
    print ("\t",p.identifier," \t| ",p.name)

    

Proyectos:
	 base  	|  Base
	 reqtest  	|  ReqTest
	 robot  	|  Robot
	 rqdemo  	|  RqDemo


In [59]:
my_project = redmine.project.get(req_project_id_str)
print ("Obtenemos proyecto: ",my_project.identifier," | ",my_project.name)

Obtenemos proyecto:  rqdemo  |  RqDemo


In [60]:
tmp = redmine.issue.filter(project_id=req_project_id_str, tracker_id=req_rq_tracker_id)
my_project_issues = sorted(tmp, key=lambda k: k.subject)

Ahora recorremos el proyecto y sacamos los diagramas completos de jerarquía y dependencias, y guardamos los ficheros de esos diagramas en la carpeta doc.

In [61]:
from graphviz import Digraph

prj_graph = Digraph(name="reporting/doc/img/"+my_project.identifier+"_h", format='svg', node_attr={'shape':'box', 'style':'filled','URL':req_server_url})
prj_graphb = Digraph(name="reporting/doc/img/"+my_project.identifier+"_d", format='svg', engine='dot', node_attr={'shape':'box', 'style':'filled','URL':req_server_url})

for i in my_project_issues:
    prj_graph.node(str(i.id),i.subject,URL=req_server_url+'/issues/'+str(i.id))
    print(i.id,": ",i.subject)
    for child in i.children:
        prj_graph.edge(str(i.id),str(child.id))
    
    my_issue_relations = redmine.issue_relation.filter(issue_id=i.id)
    #print(len(my_issue_relations))
    my_filtered_issue_relations = list(filter(lambda x: x.issue_to_id != i.id, my_issue_relations))
    #print(len(my_filtered_issue_relations))
    if (len(my_issue_relations)>0):
        prj_graphb.node(str(i.id),i.subject,URL=req_server_url+'/issues/'+str(i.id))
        for r in my_filtered_issue_relations:
            related_element = redmine.issue.get(r.issue_to_id)
            print("related_element: ",related_element," : ",related_element.tracker)
            if (related_element.tracker.id == req_rq_tracker_id):
                #print("\t"+r.relation_type+"\t"+str(r.issue_id)+"\t"+str(r.issue_to_id))
                prj_graphb.edge(str(i.id),str(r.issue_to_id),color="blue")
            

prj_graph.render()
prj_graphb.render()

print("Acabamos")

274 :  SR0001
275 :  SR0002
276 :  SR0003
277 :  SR0004
related_element:  SRC0005  :  Req
278 :  SR0005
related_element:  SR0007  :  Req
related_element:  SR0008  :  Req
279 :  SR0006
related_element:  SR0007  :  Req
280 :  SR0007
281 :  SR0008
282 :  SR0009
283 :  SR0010
284 :  SRC0001
285 :  SRC0002
286 :  SRC0003
287 :  SRC0004
288 :  SRC0005
related_element:  SRC0007  :  Req
related_element:  SRC0008  :  Req
289 :  SRC0006
related_element:  SRC0007  :  Req
290 :  SRC0007
291 :  SRC0008
292 :  SRC0009
293 :  SRC0010
Acabamos


Ahora vamos a generar los diagramas de jerarquía y de dependencia para cada una de los requisitos, y los guardaremos en la carpeta doc.

In [62]:
from lib.csysrq_support import *

# Generamos los diagramas correspondientes a los requisitos del proyecto
for my_issue in my_project_issues:
    target_issue_id = my_issue.id
    prj_graphc = Digraph(name="./reporting/doc/img/"+my_issue.subject+"_h", format='svg', engine='dot', node_attr={'shape':'box', 'style':'filled','URL':req_server_url})
    target_issue = draw_descendants(redmine,req_server_url,target_issue_id,prj_graphc)
    current_parent = getattr(target_issue, 'parent', None)
    if current_parent is not None:    
        draw_ancestors(redmine,req_server_url,target_issue.parent,target_issue_id,prj_graphc)

    prj_graphc.node(str(target_issue.id),target_issue.subject,URL=req_server_url+'/issues/'+str(target_issue.id),color='green')
    prj_graphc.render()

    print("\n\n---------- Diagrama ----------",my_issue)
    prj_graphd = Digraph(name="./reporting/doc/img/"+my_issue.subject+"_d", format='svg', engine='dot', node_attr={'shape':'box', 'style':'filled','URL':req_server_url})
    my_issue = draw_postpropagation(redmine,req_server_url,target_issue_id,prj_graphd,req_rq_tracker_id)
    if (my_issue.tracker.id == req_rq_tracker_id):
        draw_prepropagation(redmine,req_server_url,target_issue_id,prj_graphd,req_rq_tracker_id)
        prj_graphd.node(str(my_issue.id),my_issue.subject,URL=req_server_url+'/issues/'+str(my_issue.id),color='green')
        prj_graphd.render()
    
print("Acabamos")



---------- Diagrama ---------- SR0001
* Node post SR0001
* Node pre SR0001
len(my_filtered_issue_relations):  1
related pre SR


---------- Diagrama ---------- SR0002
* Node post SR0002
* Node pre SR0002
len(my_filtered_issue_relations):  1
related pre SR


---------- Diagrama ---------- SR0003
* Node post SR0003
* Node pre SR0003
len(my_filtered_issue_relations):  1
related pre SR


---------- Diagrama ---------- SR0004
* Node post SR0004
* Node post SRC0005
* Node post SRC0007
* Node post SRC0008
* Node pre SR0004
len(my_filtered_issue_relations):  1
related pre SR


---------- Diagrama ---------- SR0005
* Node post SR0005
* Node post SR0007
* Node post SR0008
* Node pre SR0005
len(my_filtered_issue_relations):  1
related pre SR


---------- Diagrama ---------- SR0006
* Node post SR0006
* Node post SR0007
* Node pre SR0006
len(my_filtered_issue_relations):  1
related pre SR


---------- Diagrama ---------- SR0007
* Node post SR0007
* Node pre SR0007
len(my_filtered_issue_relations)

Vamos a grabar el fichero JSON intermedio para generar los reportes

In [63]:
import json

# Preparamos el fichero JSON que usaremos de puente para generar la documentación

data = {}
data['docs'] = []
data['issues'] = []

tmp = redmine.issue.filter(project_id=req_project_id_str, tracker_id=req_doc_tracker_id)
my_project_docs = sorted(tmp, key=lambda k: k.subject)

for my_doc in my_project_docs:
    json_issue = {
            'id': my_doc.id,
            'subject': my_doc.subject,
            'description': my_doc.description,
            'title': my_doc.custom_fields.get(req_title_cf_id).value,
            'docprefix': my_doc.custom_fields.get(req_prefix_cf_id).value,
            'chapter': my_doc.custom_fields.get(req_chapter_cf_id).value,
    }
    data['docs'].append(json_issue)

for i in my_project_issues:
    my_issue = redmine.issue.get(i.id)
    print(dir(my_issue))
    s = getattr(my_issue, 'status', None)
    if s is not None:
        status_name = s.name
    v = getattr(my_issue, 'fixed_version', None)
    if v is not None:
        target_name = v.name
    else:
        target_name = None
        
    
    json_issue = {
            'id': my_issue.id,
            'subject': my_issue.subject,
            'description': my_issue.description,
            'title': my_issue.custom_fields.get(req_title_cf_id).value,
            'type': my_issue.custom_fields.get(req_type_cf_id).value,
            'level': my_issue.custom_fields.get(req_level_cf_id).value,
            'sources': my_issue.custom_fields.get(req_sources_cf_id).value,
            'rationale': my_issue.custom_fields.get(req_rationale_cf_id).value,
            'rq_value': my_issue.custom_fields.get(req_value_cf_id).value,
            'rq_var': my_issue.custom_fields.get(req_var_cf_id).value,
            'chapter': my_issue.custom_fields.get(req_chapter_cf_id).value,
            'target': target_name,
            'state': status_name,
            'url_h': "./img/"+my_issue.subject+"_h.gv.svg",
            'url_d': "./img/"+my_issue.subject+"_d.gv.svg"
    }
    data['issues'].append(json_issue)
    
    
    
with open('./reporting/doc/reqs.json', 'w') as outfile:  
    json.dump(data, outfile)

['attachments', 'author', 'changesets', 'children', 'closed_on', 'created_on', 'custom_fields', 'description', 'done_ratio', 'due_date', 'estimated_hours', 'fixed_version', 'id', 'is_private', 'journals', 'priority', 'project', 'relations', 'start_date', 'status', 'subject', 'time_entries', 'total_estimated_hours', 'tracker', 'updated_on', 'watchers']
['attachments', 'author', 'changesets', 'children', 'closed_on', 'created_on', 'custom_fields', 'description', 'done_ratio', 'due_date', 'estimated_hours', 'fixed_version', 'id', 'is_private', 'journals', 'parent', 'priority', 'project', 'relations', 'start_date', 'status', 'subject', 'time_entries', 'total_estimated_hours', 'tracker', 'updated_on', 'watchers']
['attachments', 'author', 'changesets', 'children', 'closed_on', 'created_on', 'custom_fields', 'description', 'done_ratio', 'due_date', 'estimated_hours', 'fixed_version', 'id', 'is_private', 'journals', 'priority', 'project', 'relations', 'start_date', 'status', 'subject', 'time_

Lanzamos la herramienta Carbone en Node, para generar los reportes de documentación.

In [64]:
from Naked.toolshed.shell import execute_js

success = execute_js('./reporting/lib/launch_carbone.js')
print(success)

if success:
    # handle success of the JavaScript
    print("Todo fue bien")

else:
    # handle failure of the JavaScript
    print("todo fue mal")

True
Todo fue bien


Vamos a generar el archivo JSON para crear el árbol

In [65]:
import json

# Preparamos el fichero JSON que usaremos para el árbol

def create_tree(current_issue):
    print("issue: " + current_issue.subject)
    tree_node = {'title': current_issue.custom_fields.get(req_chapter_cf_id).value + ": " + current_issue.subject + ": " + current_issue.custom_fields.get(req_title_cf_id).value,
             'subtitle': current_issue.description,
             'expanded': True,
             'children': [],
            }
    for c in current_issue.children:
        child_issue = redmine.issue.get(c.id)
        child_node = create_tree(child_issue)
        tree_node['children'].append(child_node)
        
    return tree_node
    
    
    
    
'''    for r in current_issue.related_issues:
        if (r.issue_to_id != current_issue.id):
            child_issue = redmine.issue.get(r.issue_to_id)
            child_node = create_tree(child_issue)
            tree_node['children'].append(child_node)
'''        
        



treedata = []
for i in my_project_issues:
    print(dir(i))
    current_parent = getattr(i, 'parent', None)
    if (current_parent == None):
        tree_node = create_tree(i)
        treedata.append(tree_node)


with open('./reqtree/src/reqtreedata.json', 'w') as outfile:  
    json.dump(treedata, outfile)


['attachments', 'author', 'changesets', 'children', 'closed_on', 'created_on', 'custom_fields', 'description', 'done_ratio', 'due_date', 'estimated_hours', 'fixed_version', 'id', 'is_private', 'journals', 'priority', 'project', 'relations', 'start_date', 'status', 'subject', 'time_entries', 'tracker', 'updated_on', 'watchers']
issue: SR0001
issue: SR0002
['attachments', 'author', 'changesets', 'children', 'closed_on', 'created_on', 'custom_fields', 'description', 'done_ratio', 'due_date', 'estimated_hours', 'fixed_version', 'id', 'is_private', 'journals', 'parent', 'priority', 'project', 'relations', 'start_date', 'status', 'subject', 'time_entries', 'tracker', 'updated_on', 'watchers']
['attachments', 'author', 'changesets', 'children', 'closed_on', 'created_on', 'custom_fields', 'description', 'done_ratio', 'due_date', 'estimated_hours', 'fixed_version', 'id', 'is_private', 'journals', 'priority', 'project', 'relations', 'start_date', 'status', 'subject', 'time_entries', 'tracker', '